To formulate this problem as a linear programming problem, we need to define the decision variables, the objective function, and the constraints.

Let's start by defining the decision variables. We need to decide how many units of t-shirts and jeans to produce in each production center. We can represent this decision with the following variables:

x_1: number of t-shirts to produce in Beşiktaş

x_2: number of jeans to produce in Beşiktaş

x_3: number of t-shirts to produce in Davutpaşa

x_4: number of jeans to produce in Davutpaşa

Now we can define the objective function. The objective is to maximize the profit, which is equal to the total revenue from the sale of t-shirts and jeans. The total revenue is equal to the price per unit of each product multiplied by the number of units produced. Therefore, the objective function can be defined as:

maximize 10x_1 + 15x_2 + 10x_3 + 15x_4

The coefficient of each decision variable represents the contribution to the profit of producing one unit of the corresponding product.

Finally, we need to define the constraints. The first constraint is that the total production in each center must not exceed the capacity of the cutting and sewing processes. We can represent this constraint as:

x_1\*[0][0][0] + x_2\*[0][0][1] <= 80 (cutting capacity in Beşiktaş)

x_1\*[1][0][0] + x_2\[1][0][1] <= 60 (sewing capacity in Beşiktaş)

x_3\*[0][1][0] + x_4\*[0][1][1] <= 60 (cutting capacity in Davutpaşa)

x_3\*[1][1][0] + x_4\*[1][1][1] <= 75 (sewing capacity in Davutpaşa)

where the time it takes to cut and sew one unit of each product in each center is given in the table below:

constraints[i][j][k]

i: index of the row (0: Cutting, 1: Sewing)

j: index of the column (0: Beşiktaş, 1: Davutpaşa)

k: index of the element within the column (0: T-shirts, 1: Jeans)


The second constraint is that the total demand for raw material must not exceed the available supply. We can represent this constraint as:

4*(x_1 + x_2 + x_3 + x_4) <= 120 (cloth supply)

Now that we have defined all the necessary components, we can use PuLP to solve this problem.

Here is the code that sets up and solves the problem using CBC:

In [62]:
import pandas as pd
import numpy as np
import scipy
import pulp

In [63]:
np.random.seed(21506050)

In [64]:
# Requirment of the production (proudction hour)
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 4),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (2,2))
BesiktasHour = pd.DataFrame(RandomFtoD2, columns=['T-shirts', 'Jeans'], 
                        index=["Cutting", "Sewing"])
BesiktasHour.columns=pd.MultiIndex.from_product([['Beşiktaş'],BesiktasHour.columns])

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 4),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (2,2))
DavutpasaHour = pd.DataFrame(RandomDtoS2, columns=['T-shirts', 'Jeans'], 
                        index=["Cutting", "Sewing"])
DavutpasaHour.columns=pd.MultiIndex.from_product([['Davutpaşa'],DavutpasaHour.columns])

# Your Required Information About Production Hour
production_hours = pd.concat([BesiktasHour, DavutpasaHour], axis=1)
print(production_hours)

        Beşiktaş       Davutpaşa      
        T-shirts Jeans  T-shirts Jeans
Cutting       10     4         2     7
Sewing         9    10         7     8


In [65]:
# Extract the constraints from the production_hours dataframe
constraints = []
for i, row in enumerate(production_hours.iterrows()):
    constraint = []
    for j, col in enumerate(production_hours.columns):
        if j % 2 == 0:
            sub_constraint = []
        sub_constraint.append(row[1][col])
        if j % 2 == 1:
            constraint.append(sub_constraint)
    constraints.append(constraint)

In [66]:
# Create the linear programming model
model = pulp.LpProblem("Profit_Maximization", pulp.LpMaximize)

In [67]:
# Define the decision variables
x_1 = pulp.LpVariable('x_1', lowBound=0, cat='Integer')
x_2 = pulp.LpVariable('x_2', lowBound=0, cat='Integer')
x_3 = pulp.LpVariable('x_3', lowBound=0, cat='Integer')
x_4 = pulp.LpVariable('x_4', lowBound=0, cat='Integer')

In [68]:
# Define the objective function
model += 10*x_1 + 15*x_2 + 10*x_3 + 15*x_4

In [69]:
# Define the constraints
model += x_1*constraints[0][0][0] + x_2*constraints[0][0][1] <= 80
model += x_1*constraints[1][0][0] + x_2*constraints[1][0][1] <= 60
model += x_3*constraints[0][1][0] + x_4*constraints[0][1][1] <= 60
model += x_3*constraints[1][1][0] + x_4*constraints[1][1][1] <= 75
model += 4*(x_1 + x_2 + x_3 + x_4) <= 120

In [70]:
# Solve the linear programming model using CBC
feasible = model.solve(solver=pulp.COIN_CMD(msg=0))

In [71]:
# Print the results
if feasible == 1:
    print("Production Quantities:")
    print("Jeans in Beşiktaş: {}".format(x_2.value()))
    print("Jeans in Davutpaşa: {}".format(x_4.value()))
    print("T-shirts in Beşiktaş: {}".format(x_1.value()))
    print("T-shirts in Davutpaşa: {}".format(x_3.value()))
    print("Total amount used:")
    print("Cutting in Beşiktaş: {}".format(x_1.value() + x_2.value()))
    print("Cutting in Davutpaşa: {}".format(x_3.value() + x_4.value()))
    print("Sewing in Beşiktaş: {}".format(x_1.value() + x_2.value()))
    print("Sewing in Davutpaşa: {}".format(x_3.value() + x_4.value()))
    print("Cloths in Beşiktaş: {}".format(4*(x_1.value() + x_2.value())))
    print("Cloths in Davutpaşa: {}".format(4*(x_3.value() + x_4.value())))
    print("Total profit: {}".format(pulp.value(model.objective)))
else:
    print("No feasible solution found")

Production Quantities:
Jeans in Beşiktaş: 6.0
Jeans in Davutpaşa: 8.0
T-shirts in Beşiktaş: 0.0
T-shirts in Davutpaşa: 1.0
Total amount used:
Cutting in Beşiktaş: 6.0
Cutting in Davutpaşa: 9.0
Sewing in Beşiktaş: 6.0
Sewing in Davutpaşa: 9.0
Cloths in Beşiktaş: 24.0
Cloths in Davutpaşa: 36.0
Total profit: 220.0
